In [ ]:
import numpy as np
import torch
import os
import pickle
import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams['figure.figsize'] = (20, 20)
# plt.rcParams['image.interpolation'] = 'bilinear'

import sys
sys.path.append('../train/')

from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn

import collections
import numbers
import random
import math
from PIL import Image, ImageOps, ImageEnhance
import time
from torch.utils.data import Dataset

from networks.UNet_check import UNet_check
import tool
from tqdm import tqdm

flip_index = ['16', '15', '14', '13', '12', '11', '10']

In [ ]:
NUM_CHANNELS = 3
NUM_CLASSES = 2 
BATCH_SIZE = 8
W, H = 1918, 1280
STRIDE = 388
IMAGE_SIZE = 572
test_mask_path = '../../data/test_masks/UNet/'
weight_path = '../_weights/UNet_check-fold0-0.00549.pth'

In [ ]:
def load_model(filename, model):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state'])

In [ ]:
model = UNet_check()
model = model.cuda()
model.eval()
load_model(weight_path, model)

In [ ]:
test_path = '../../data/images/test/'

if not os.path.exists(test_mask_path):
    os.makedirs(test_mask_path)

In [ ]:
test_names = os.listdir(test_path)
test_names = sorted(test_names)

In [ ]:
# coords_full = np.zeros((BATCH_SIZE, 2, H, W), dtype='float32')
# xx,yy = np.meshgrid(np.linspace(-0.5,0.5,W), np.linspace(-0.5,0.5,H))
# coord = np.concatenate([xx[np.newaxis,...], yy[np.newaxis,...]],0).astype('float32')
# for i in range(BATCH_SIZE):
#     coords_full[i] = coord

In [ ]:
board = 572 - 388

In [ ]:
with torch.no_grad():
    batch_size = BATCH_SIZE
    normalize_mean = [.485, .456, .406]
    normalize_std = [.229, .224, .225]

    test_names = sorted(os.listdir(test_path))
    for image_pack in tqdm(range(len(test_names) // batch_size)):
        images = np.zeros((batch_size, 3, H + board, W + board), dtype='float32')
        test_masks = np.zeros((batch_size, 2, H, W), dtype='float32')
        ifflip = [False] * batch_size
        image_batch_names = test_names[image_pack * batch_size: image_pack * batch_size + batch_size]
        mask_names = [input_name.split('.')[0] + '.png' for input_name in image_batch_names]
        
        for idx, image_name in enumerate(image_batch_names):
            image = Image.open(os.path.join(test_path, image_name))
            angle = image_name.split('.')[0].split('_')[-1]
            if angle in flip_index:
                ifflip[idx] = True
                image = ImageOps.mirror(image)

            image = np.array(image).astype('float') / 255
            image = image.transpose(2, 0, 1)
            img = np.zeros((3, H + board, W + board), dtype='float32')
            img[:, board // 2:-board // 2, board // 2:-board // 2] = image
            image = img

            for i in range(3):
                image[i] = (image[i] - normalize_mean[i]) / normalize_std[i]

            images[idx] = image
            
            
#             images[idx] = image
    

        for h_idx in range(int(math.ceil((H + board - STRIDE) / STRIDE))):
            h_start = h_idx * STRIDE
            h_end = h_start + 572
            
            mask_h_start = h_idx * STRIDE
            mask_h_end = mask_h_start + 388
            
            if h_end > H + board:
                h_end = H + board
                h_start = h_end - 572
                
                mask_h_end = H
                mask_h_start = mask_h_end - 388
                
            for w_idx in range(int(math.ceil((W + board - STRIDE) / STRIDE))):
                w_start = w_idx * STRIDE
                w_end = w_start + 572
                
                mask_w_start = w_idx * STRIDE
                mask_w_end = mask_w_start + 388
                
                if w_end > W + board:
                    w_end = W + board
                    w_start = w_end - 572
                    
                    mask_w_end = W
                    mask_w_start = mask_w_end - 388

                input_batchs = images[:, :, h_start:h_end, w_start:w_end]
                input_tensor = torch.from_numpy(input_batchs).cuda()
                inputs = Variable(input_tensor)
                
                
                outputs = model(inputs)
                ouputs = outputs.cpu().data.numpy()

                test_masks[:, :, mask_h_start:mask_h_end, mask_w_start:mask_w_end] += ouputs
        
    
        test_masks = np.argmax(test_masks, axis=1).astype('uint8')
        for idx in range(batch_size):
            output_PIL = Image.fromarray(test_masks[idx].astype('uint8')*255).convert('1')
            if ifflip[idx]:
                output_PIL = ImageOps.mirror(output_PIL)
            mask_name = mask_names[idx]
            output_PIL.save(test_mask_path + mask_name)
